In [3]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [4]:
# 2.a - Read frames from video

def video_to_frames(vid_path: str, start_second, end_second):
    """
    Load a video and return its frames from the wanted time range.
    :param vid_path: video file path.
    :param start_second: time of first frame to be taken from the
    video in seconds.
    :param end_second: time of last frame to be taken from the
    video in seconds.
    :return:
    frame_set: a 4D uint8 np array of size [num_of_frames x H x W x C]
    containing the wanted video frames.
    """

    #calculate desired frames 
    cap = cv2.VideoCapture(vid_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    start_frame = start_second*fps
    end_frame = end_second*fps
    num_of_frames = 1 + (end_frame-start_frame) 

    #get the desired frames
    frames = []
    curr_frame = 0
    i = 0

    while ( i<num_of_frames):
        valid, img = cap.read() # read one frame from the 'capture' object; img is (H, W, C)
        curr_fr += 1
        if (curr_frame >= start_frame): #if started to get inside the desired section, save frame
            frames.append(img)
            i += 1
    frame_set = np.array(frames) # convert to np array,dimensions (T*fps, H, W, C)
    
    cap.release()

    return frame_set

In [5]:
def match_corr(corr_obj, img):
    """
    return the center coordinates of the location of 'corr_obj' in 'img'.
    :param corr_obj: 2D numpy array of size [H_obj x W_obj]
    containing an image of a component.
    :param img: 2D numpy array of size [H_img x W_img]
    where H_img >= H_obj and W_img>=W_obj,
    containing an image with the 'corr_obj' component in it.
    :return:
    match_coord: the two center coordinates in 'img'
    of the 'corr_obj' component.
    """
 # ====== YOUR CODE: ======
 # corr_obj correltion with itself
    match_corr_obj = cv2.filter2D(corr_obj.astype('float32'), -1, corr_obj.astype('float32'), borderType=cv2.BORDER_CONSTANT)

 # maximal correlation value of the corr_obj image with itself
    max_obj = np.max(match_corr_obj) 
    
 # corr_obj correltion with itself
    img_corr_obj = cv2.filter2D(img.astype('float32'), -1, corr_obj.astype('float32'), borderType=cv2.BORDER_CONSTANT)   
    match_coord = np.unravel_index((np.abs(img_corr_obj - max_obj)).argmin(), img_corr_obj.shape)    
 

    return match_coord

In [6]:
# 2.c - Pre-Processing

frames = video_to_frames("../given_data/Corsica.mp4", 250, 260)

top_crop = int(frames[0].shape[0] / 3)
left_crop = 7
right_crop = 627

def crop_frame(frame, top_crop, left_crop, right_crop):
    cropped = frame[top_crop:, left_crop:right_crop]
    return cropped

cropped_frames = []

for frame in frames:
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cropped_frames.append(crop_frame(frame, top_crop, left_crop, right_crop))




UnboundLocalError: local variable 'curr_fr' referenced before assignment

In [ ]:
# 2.d - Creating the panorama base


cropped_frames_h, cropped_frames_w = cropped_frames[0].shape
panorama = np.zeros((cropped_frames_h, int(cropped_frames_w * 2.5)))

ref_frame = cropped_frames[int(len(cropped_frames) / 2)]

ref_right_edge = int((panorama.shape[1] - ref_frame.shape[1]) / 2 + ref_frame.shape[1])
ref_left_edge = int((panorama.shape[1] - ref_frame.shape[1]) / 2)
panorama[:, ref_left_edge:ref_right_edge] = ref_frame

plt.imshow(ref_frame, cmap='gray')
plt.title("original reference frame")
plt.show()

plt.imshow(panorama, cmap='gray')
plt.title("Panorama image - with the reference frame in the center")
plt.show()

ref_index = int(len(cropped_frames) / 2)
early_frame = cropped_frames[ref_index - 70]
late_frame = cropped_frames[ref_index + 70]

plt.imshow(early_frame, cmap='gray')
plt.title("early frame")
plt.show()

plt.imshow(late_frame, cmap='gray')
plt.title("late frame")
plt.show()


In [ ]:
# 2.e - Frames matching

sub_early_frame = early_frame[:,:300]
sub_late_frame = late_frame[:, 400:]


early_match_coord = match_corr(sub_early_frame, ref_frame)
late_match_coord = match_corr(sub_late_frame, ref_frame)

plt.imshow(sub_early_frame, cmap='gray')
plt.title(early_match_coord)
plt.show()

plt.imshow(sub_late_frame, cmap='gray')
plt.title(late_match_coord)
plt.show()

In [ ]:
#2.f

# Calculate the bounds for the left side of the panorama
late_overlap_right_panorama =int(ref_left_edge + late_match_coord[1] + int((sub_late_frame.shape[1])/2))
late_overlap_left_panorama = int(ref_left_edge)
late_overlap_left = late_frame.shape[1] - (late_overlap_right_panorama - late_overlap_left_panorama)
late_start_panorama = late_overlap_right_panorama - late_frame.shape[1]

# Calculate the bounds for the right side of the panorama
early_overlap_right_panorama = ref_right_edge
early_overlap_left_panorama = int(ref_left_edge + early_match_coord[1] - int((sub_early_frame.shape[1])/2))
early_overlap_right = early_overlap_right_panorama - early_overlap_left_panorama
early_end_panorama = early_overlap_left_panorama + early_frame.shape[1]

#making the left side panorama
panorama[:, late_overlap_left_panorama:late_overlap_right_panorama] = ((panorama[:, late_overlap_left_panorama:late_overlap_right_panorama] + late_frame[:,late_overlap_left:])/2)
panorama[:, late_start_panorama:late_overlap_left_panorama] = late_frame[:,:late_overlap_left]

#making the right side panorama
panorama[:, early_overlap_left_panorama:early_overlap_right_panorama] = ((panorama[:, early_overlap_left_panorama:early_overlap_right_panorama] + early_frame[:, :early_overlap_right])/2)
panorama[:, ref_right_edge:early_end_panorama] = early_frame[:,early_overlap_right:]


plt.imshow(panorama, cmap='gray')
plt.title("final panorama")
plt.show()